In [1]:
import os
import os.path as path
import boto3
import botocore

from csv import writer

import pandas as pd

In [6]:
project_root = os.getcwd()

data_dir = 'data'
log_file_name = 'BGL.log'

input_dir = 'input'
log_csv_file_name = 'BGL.csv'

bucket_name = 'sagemaker-studio-326787221562-jycpwz9gs3f'

In [12]:
log_csv_file = log_file = path.abspath(path.join(project_root, '../', input_dir, log_csv_file_name))
if os.path.exists(log_csv_file):
    os.remove(log_csv_file)

log_file = path.abspath(path.join(project_root, '../', data_dir, log_file_name))
if os.path.exists(log_file):
    os.remove(log_file)

s3 = boto3.resource('s3')
try:
    s3.Bucket(bucket_name).download_file(log_file_name, log_file)
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == "404":
        print("The object does not exist.")
    else:
        raise

In [ ]:
logs = open(log_file, 'r')

original_dataset_len = 0
with open(log_csv_file, 'a') as log_csv_file_obj:
    log_csv_writer_obj = writer(log_csv_file_obj)
    log_csv_writer_obj.writerow(['Anomaly Type', 'Timestamp (ms)', 'Date', 'Node', 'Timestamp', 'Node Repeat', 'Message Type', 'Component', 'Level', 'Content'])
    for line in logs:
        original_dataset_len += 1
        split_data = line.rstrip('\n').split(' ')
        if len(split_data) > 9:
            split_data[9] = ' '.join(split_data[9:])
            log_csv_writer_obj.writerow(split_data[0:10])
    log_csv_file_obj.close

In [ ]:
df = pd.read_csv(log_csv_file)
processed_dataset_len = len(df)

print('Number of rows without content (rejected) - ', original_dataset_len - processed_dataset_len)

In [ ]:
client = boto3.client("s3")
client.upload_file(log_csv_file, bucket_name, log_csv_file_name)